## 关于学生叙述能力的差异分析

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway, tukey_hsd, levene, shapiro
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 设置中文字体和图表样式
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")
plt.style.use('seaborn-v0_8')

In [3]:
def load_and_prepare_data():
    """加载和准备数据"""
    # 读取数据
    data = pd.read_csv('../pre_data/描述性统计-grade.csv')
    
    # 将年级标签转换为更易理解的名称
    grade_mapping = {1: '高一', 2: '高二', 3: '高三'}
    data['年级名称'] = data['年级'].map(grade_mapping)
    
    # 按年级分组数据
    grade_groups = {grade: group for grade, group in data.groupby('年级名称')}
    
    return data, grade_groups

In [4]:
def descriptive_statistics(data, grade_groups):
    """描述性统计分析"""
    print("=" * 60)
    print("描述性统计分析报告")
    print("=" * 60)
    
    # 1. 基本统计信息
    print("\n1. 数据基本信息：")
    print(f"总样本数：{len(data)}")
    print(f"年级分布：\n{data['年级名称'].value_counts().sort_index()}")
    
    # 2. 各年级描述性统计
    print("\n2. 各年级描述性统计：")
    
    for grade_name in ['高一', '高二', '高三']:
        group_data = grade_groups[grade_name]
        print(f"\n{grade_name}：")
        print(f"  样本数：{len(group_data)}")
        print(f"  单词数量 - 均值：{group_data['单词数量-DESWC-03'].mean():.2f}, 标准差：{group_data['单词数量-DESWC-03'].std():.2f}")
        print(f"  句子数量 - 均值：{group_data['句子数量-DESSC-02'].mean():.2f}, 标准差：{group_data['句子数量-DESSC-02'].std():.2f}")

In [5]:
def normality_test(data, grade_groups):
    """正态性检验"""
    print("\n3. 正态性检验：")
    
    for grade_name in ['高一', '高二', '高三']:
        group_data = grade_groups[grade_name]
        
        # 单词数量正态性检验
        stat_word, p_word = shapiro(group_data['单词数量-DESWC-03'])
        # 句子数量正态性检验
        stat_sent, p_sent = shapiro(group_data['句子数量-DESSC-02'])
        
        print(f"\n{grade_name}：")
        print(f"  单词数量 - Shapiro-Wilk检验：W={stat_word:.4f}, p={p_word:.4f} {'(正态分布)' if p_word > 0.05 else '(非正态分布)'}")
        print(f"  句子数量 - Shapiro-Wilk检验：W={stat_sent:.4f}, p={p_sent:.4f} {'(正态分布)' if p_sent > 0.05 else '(非正态分布)'}")

In [6]:
def variance_homogeneity_test(data, grade_groups):
    """方差齐性检验"""
    print("\n4. 方差齐性检验：")
    
    # 提取各年级数据
    grade1_words = grade_groups['高一']['单词数量-DESWC-03']
    grade2_words = grade_groups['高二']['单词数量-DESWC-03']
    grade3_words = grade_groups['高三']['单词数量-DESWC-03']
    
    grade1_sents = grade_groups['高一']['句子数量-DESSC-02']
    grade2_sents = grade_groups['高二']['句子数量-DESSC-02']
    grade3_sents = grade_groups['高三']['句子数量-DESSC-02']
    
    # Levene检验
    stat_word, p_word = levene(grade1_words, grade2_words, grade3_words)
    stat_sent, p_sent = levene(grade1_sents, grade2_sents, grade3_sents)
    
    print(f"单词数量 - Levene检验：F={stat_word:.4f}, p={p_word:.4f} {'(方差齐性)' if p_word > 0.05 else '(方差非齐性)'}")
    print(f"句子数量 - Levene检验：F={stat_sent:.4f}, p={p_sent:.4f} {'(方差齐性)' if p_sent > 0.05 else '(方差非齐性)'}")

In [12]:

def grade_comparison_analysis(data, grade_groups):
    """年级间比较分析"""
    print("\n5. 年级间比较分析：")
    
    # 提取各年级数据
    grade1_words = grade_groups['高一']['单词数量-DESWC-03']
    grade2_words = grade_groups['高二']['单词数量-DESWC-03']
    grade3_words = grade_groups['高三']['单词数量-DESWC-03']
    
    grade1_sents = grade_groups['高一']['句子数量-DESSC-02']
    grade2_sents = grade_groups['高二']['句子数量-DESSC-02']
    grade3_sents = grade_groups['高三']['句子数量-DESSC-02']
    
    # 单词数量分析
    print("\n单词数量年级间比较：")
    f_stat_word, p_word = f_oneway(grade1_words, grade2_words, grade3_words)
    print(f"  单因素方差分析：F={f_stat_word:.4f}, p={p_word:.4f}")
    
    if p_word < 0.05:
        print("  结果：存在显著性差异")
        # Tukey HSD事后检验
        tukey_result = tukey_hsd(grade1_words, grade2_words, grade3_words)
        print("  Tukey HSD事后检验结果：")
        # 修复：提取数组中的具体数值
        print(f"    高一 vs 高二：p={float(tukey_result.pvalue[0]):.4f}")
        print(f"    高一 vs 高三：p={float(tukey_result.pvalue[1]):.4f}")
        print(f"    高二 vs 高三：p={float(tukey_result.pvalue[2]):.4f}")
    else:
        print("  结果：无显著性差异")
    
    # 句子数量分析
    print("\n句子数量年级间比较：")
    f_stat_sent, p_sent = f_oneway(grade1_sents, grade2_sents, grade3_sents)
    print(f"  单因素方差分析：F={f_stat_sent:.4f}, p={p_sent:.4f}")
    
    if p_sent < 0.05:
        print("  结果：存在显著性差异")
        # Tukey HSD事后检验
        tukey_result = tukey_hsd(grade1_sents, grade2_sents, grade3_sents)
        print("  Tukey HSD事后检验结果：")
        # 修复：提取数组中的具体数值
        print(f"    高一 vs 高二：p={float(tukey_result.pvalue[0]):.4f}")
        print(f"    高一 vs 高三：p={float(tukey_result.pvalue[1]):.4f}")
        print(f"    高二 vs 高三：p={float(tukey_result.pvalue[2]):.4f}")
    else:
        print("  结果：无显著性差异")

In [7]:
def within_grade_analysis(data, grade_groups):
    """年级内比较分析"""
    print("\n6. 年级内比较分析：")
    
    for grade_name in ['高一', '高二', '高三']:
        group_data = grade_groups[grade_name]
        
        print(f"\n{grade_name}内部差异分析：")
        
        # 单词数量内部差异
        word_cv = group_data['单词数量-DESWC-03'].std() / group_data['单词数量-DESWC-03'].mean() * 100
        word_range = group_data['单词数量-DESWC-03'].max() - group_data['单词数量-DESWC-03'].min()
        word_iqr = group_data['单词数量-DESWC-03'].quantile(0.75) - group_data['单词数量-DESWC-03'].quantile(0.25)
        
        print(f"  单词数量：")
        print(f"    变异系数：{word_cv:.2f}%")
        print(f"    极差：{word_range}")
        print(f"    四分位距：{word_iqr:.2f}")
        
        # 句子数量内部差异
        sent_cv = group_data['句子数量-DESSC-02'].std() / group_data['句子数量-DESSC-02'].mean() * 100
        sent_range = group_data['句子数量-DESSC-02'].max() - group_data['句子数量-DESSC-02'].min()
        sent_iqr = group_data['句子数量-DESSC-02'].quantile(0.75) - group_data['句子数量-DESSC-02'].quantile(0.25)
        
        print(f"  句子数量：")
        print(f"    变异系数：{sent_cv:.2f}%")
        print(f"    极差：{sent_range}")
        print(f"    四分位距：{sent_iqr:.2f}")

In [8]:
def create_visualizations(data, grade_groups):
    """创建可视化图表"""
    print("\n7. 生成可视化图表...")
    
    # 创建图形
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('学生叙述能力描述性统计分析', fontsize=16, fontweight='bold')
    
    # 1. 单词数量箱线图
    word_data = [grade_groups['高一']['单词数量-DESWC-03'], 
                 grade_groups['高二']['单词数量-DESWC-03'], 
                 grade_groups['高三']['单词数量-DESWC-03']]
    axes[0, 0].boxplot(word_data, labels=['高一', '高二', '高三'])
    axes[0, 0].set_title('单词数量年级间比较')
    axes[0, 0].set_ylabel('单词数量')
    
    # 2. 句子数量箱线图
    sent_data = [grade_groups['高一']['句子数量-DESSC-02'], 
                 grade_groups['高二']['句子数量-DESSC-02'], 
                 grade_groups['高三']['句子数量-DESSC-02']]
    axes[0, 1].boxplot(sent_data, labels=['高一', '高二', '高三'])
    axes[0, 1].set_title('句子数量年级间比较')
    axes[0, 1].set_ylabel('句子数量')
    
    # 3. 单词数量密度图
    for grade_name in ['高一', '高二', '高三']:
        grade_data = grade_groups[grade_name]['单词数量-DESWC-03']
        axes[0, 2].hist(grade_data, alpha=0.6, label=grade_name, bins=15, density=True)
    axes[0, 2].set_title('单词数量分布密度图')
    axes[0, 2].set_xlabel('单词数量')
    axes[0, 2].set_ylabel('密度')
    axes[0, 2].legend()
    
    # 4. 句子数量密度图
    for grade_name in ['高一', '高二', '高三']:
        grade_data = grade_groups[grade_name]['句子数量-DESSC-02']
        axes[1, 0].hist(grade_data, alpha=0.6, label=grade_name, bins=15, density=True)
    axes[1, 0].set_title('句子数量分布密度图')
    axes[1, 0].set_xlabel('句子数量')
    axes[1, 0].set_ylabel('密度')
    axes[1, 0].legend()
    
    # 5. 散点图：单词数量 vs 句子数量
    colors = ['red', 'blue', 'green']
    for i, grade_name in enumerate(['高一', '高二', '高三']):
        grade_data = grade_groups[grade_name]
        axes[1, 1].scatter(grade_data['单词数量-DESWC-03'], 
                          grade_data['句子数量-DESSC-02'], 
                          alpha=0.6, label=grade_name, color=colors[i])
    axes[1, 1].set_title('单词数量与句子数量关系')
    axes[1, 1].set_xlabel('单词数量')
    axes[1, 1].set_ylabel('句子数量')
    axes[1, 1].legend()
    
     # 6. 均值比较条形图
    word_means = [grade_groups['高一']['单词数量-DESWC-03'].mean(),
                  grade_groups['高二']['单词数量-DESWC-03'].mean(),
                  grade_groups['高三']['单词数量-DESWC-03'].mean()]
    sent_means = [grade_groups['高一']['句子数量-DESSC-02'].mean(),
                  grade_groups['高二']['句子数量-DESSC-02'].mean(),
                  grade_groups['高三']['句子数量-DESSC-02'].mean()]
    
    x = np.arange(3)
    width = 0.35
    
    axes[1, 2].bar(x - width/2, word_means, width, label='单词数量', alpha=0.8)
    axes[1, 2].bar(x + width/2, sent_means, width, label='句子数量', alpha=0.8)
    axes[1, 2].set_title('各年级均值比较')
    axes[1, 2].set_xlabel('年级')
    axes[1, 2].set_ylabel('数量')
    axes[1, 2].set_xticks(x)
    axes[1, 2].set_xticklabels(['高一', '高二', '高三'])
    axes[1, 2].legend()
    
    plt.tight_layout()
    plt.savefig('../res/描述性统计分析结果.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("图表已保存为 '../res/描述性统计分析结果.png'")

In [13]:
def main():
    """主函数"""
    # 加载数据
    data, grade_groups = load_and_prepare_data()
    
    # 执行各项分析
    descriptive_statistics(data, grade_groups)
    normality_test(data, grade_groups)
    variance_homogeneity_test(data, grade_groups)
    grade_comparison_analysis(data, grade_groups)
    within_grade_analysis(data, grade_groups)
    
    # 创建可视化
    create_visualizations(data, grade_groups)
    
    print("\n" + "=" * 60)
    print("描述性统计分析完成！")
    print("=" * 60)

if __name__ == "__main__":
    main()

描述性统计分析报告

1. 数据基本信息：
总样本数：300
年级分布：
年级名称
高一    100
高三    100
高二    100
Name: count, dtype: int64

2. 各年级描述性统计：

高一：
  样本数：100
  单词数量 - 均值：199.85, 标准差：34.64
  句子数量 - 均值：14.80, 标准差：3.17

高二：
  样本数：100
  单词数量 - 均值：181.86, 标准差：39.66
  句子数量 - 均值：13.39, 标准差：3.00

高三：
  样本数：100
  单词数量 - 均值：216.33, 标准差：31.39
  句子数量 - 均值：13.79, 标准差：2.76

3. 正态性检验：

高一：
  单词数量 - Shapiro-Wilk检验：W=0.9635, p=0.0072 (非正态分布)
  句子数量 - Shapiro-Wilk检验：W=0.9713, p=0.0277 (非正态分布)

高二：
  单词数量 - Shapiro-Wilk检验：W=0.9618, p=0.0054 (非正态分布)
  句子数量 - Shapiro-Wilk检验：W=0.9659, p=0.0108 (非正态分布)

高三：
  单词数量 - Shapiro-Wilk检验：W=0.9777, p=0.0879 (正态分布)
  句子数量 - Shapiro-Wilk检验：W=0.9648, p=0.0089 (非正态分布)

4. 方差齐性检验：
单词数量 - Levene检验：F=1.9665, p=0.1418 (方差齐性)
句子数量 - Levene检验：F=0.4580, p=0.6330 (方差齐性)

5. 年级间比较分析：

单词数量年级间比较：
  单因素方差分析：F=23.7287, p=0.0000
  结果：存在显著性差异
  Tukey HSD事后检验结果：


TypeError: only length-1 arrays can be converted to Python scalars